In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('df_train_anon.csv',header=None)
print df.shape

(1167, 550)


In [14]:
X = df.iloc[:,:549]
y = df.iloc[:,549]
print X.shape,y.shape

(1167, 549) (1167,)


In [20]:
#build a pipline
#gridsearch CV
#train X
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y.unique())

trans_y = le.transform(y)

In [284]:
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

def countaccuracy(ypre,y):
    count = 0
    total = len(y)
    for i in xrange(len(ypre)):
        if ypre[i]==y[i]:
            count+=1
        else:
            pass
    return count*1.0/total

class scalertransform(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):#,pcaparam):
        self.scaler = StandardScaler()
        #self.scaler = normalize(axis=0)
        #self.dimred = PCA(n_components=pcaparam)
        #self.le = LabelEncoder()
        self.dimred = PCA(n_components=60)
        pass
    
    def fit(self,X,y):
        #self.le.fit(y.unique())
        #temp = normalize(X)
        self.scaler.fit(X,y)
        temp = self.scaler.transform(X)
        
        
        self.dimred.fit(temp,y)
        return self
    
    def transform(self,X):
        #temp = normalize(X)
        temp = self.scaler.transform(X)
        X_trans = self.dimred.transform(temp)
        #print X_trans.shape
        return X_trans

class estimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self):
        self.est = RandomForestClassifier(n_estimators=50)
        #self.est = SVC()
        
    def fit(self,X,y):
        self.est.fit(X,y)
        return self
        
    def predict(self,X):
        return self.est.predict(X)
    
    def socre(self,X,y):
        return self.est.score(X,y)
    
pipeline = Pipeline([('scaler',scalertransform()),('estimator',estimator())])

In [276]:
#from sklearn.grid_search import GridSearchCV
#pipeline.get_params
#a=dict(scaler__pcaparam=[40,50,60])
#print a
#gscv = GridSearchCV(pipeline,param_grid=[a])
#print gscv
#pipeline.scaler.get_params().keys()

In [277]:
#gscv.fit(X,trans_y)
print type(X),type(trans_y)
X2 = X.as_matrix()
print type(X2)
pipeline.fit(X2[:800,:],trans_y[:800])

<class 'pandas.core.frame.DataFrame'> <type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(800, 60)


Pipeline(steps=[('scaler', scalertransform()), ('estimator', estimator())])

In [278]:
print X2
print pipeline.score(X2[800:,:],trans_y[800:])
#print pipeline.score(X2[:800,:],trans_y[:800])
ypre = pipeline.predict(X2[800:,:])
print type(ypre),type(trans_y[800:])
print ypre.shape,trans_y[800:].shape
print countaccuracy(ypre,trans_y[800:])

[[  8.93099e-03   8.91138e-05 ...,   1.73343e-01   9.93840e+01]
 [  9.04362e-03   1.40528e-04 ...,   5.78411e-01   1.23047e+02]
 ..., 
 [  1.73311e-01   1.16585e-02 ...,   4.72950e-01   1.20185e+02]
 [  1.76325e-01   2.42321e-03 ...,   3.02260e-01   1.01333e+02]]
(367, 60)
0.111031096563
(367, 60)
<type 'numpy.ndarray'> <type 'numpy.ndarray'>
(367,) (367,)
0.773841961853


In [286]:
import dill
pipeline.fit(X,trans_y)
with open('encoder.model','wb') as f:
    dill.dump(le,f)
f.close()
with open('predict.model','wb') as f:
    dill.dump(pipeline,f)
f.close()

In [287]:
with open('predict.model','rb') as f:
    ppp = dill.load(f)

In [297]:
result = ppp.predict([X2[0,:]])
print result[0]

2


In [300]:
print le.inverse_transform(result)

['jazz']
